# Churn analytics 
* It is the process of measuring the rate at which customers quit the product, site, or service

In [1]:
import pandas as pd
import datetime as dt

### Task
* Find out the best churned customers to retain them by distributing 1000 dollars coupen in total
* Cutoff Date = last two months

In [2]:
dataset = pd.read_csv('rfm.txt', parse_dates=['trans_date'])
dataset.head()

,customer_id,trans_date,tran_amount
0,FM5295,2017-11-11,35
1,FM4768,2019-12-15,39
2,FM2122,2017-11-26,52
3,FM1217,2016-08-16,99
4,FM1850,2018-08-20,78


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   customer_id  125000 non-null  object        
 1   trans_date   125000 non-null  datetime64[ns]
 2   tran_amount  125000 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 2.9+ MB


In [4]:
dataset['trans_date'].agg(['min', 'max'])

min   2016-02-16
max   2019-12-16
Name: trans_date, dtype: datetime64[ns]

In [5]:
cutoff_date = dt.datetime(2019, 10, 16)
print(cutoff_date)

2019-10-16 00:00:00


### Grouping data by customers

In [6]:
customer_id_groupby = dataset.groupby(['customer_id'])
customer_id_groupby.get_group('FM5295').head()

,customer_id,trans_date,tran_amount
0,FM5295,2017-11-11,35
3556,FM5295,2019-08-25,54
4218,FM5295,2018-02-22,37
4710,FM5295,2017-09-24,36
5986,FM5295,2017-12-22,98


### Identifying last arrival and max transaction amount

In [7]:
last_transection = customer_id_groupby.max()
last_transection.head()

,trans_date,tran_amount
customer_id,,
FM1112,2019-10-14,105
FM1113,2019-11-09,98
FM1114,2019-11-12,105
FM1115,2019-12-05,104
FM1116,2019-05-25,105


### Identifying churned customers by cutoff date

In [8]:
churned_customers = customer_id_groupby.max()

In [9]:
def churned(date):
    if date <= cutoff_date:
        return 1
    else:
        return 0

In [10]:
churned_customers['churned'] = churned_customers.trans_date.map(churned)
churned_customers.head()

,trans_date,tran_amount,churned
customer_id,,,
FM1112,2019-10-14,105,1
FM1113,2019-11-09,98,0
FM1114,2019-11-12,105,0
FM1115,2019-12-05,104,0
FM1116,2019-05-25,105,1


### Identifying total number of transactions by each customer

In [11]:
churned_customers['no_of_tran'] = customer_id_groupby.size()
churned_customers['tran_total'] = customer_id_groupby['tran_amount'].sum()
churned_customers.head()

,trans_date,tran_amount,churned,no_of_tran,tran_total
customer_id,,,,,
FM1112,2019-10-14,105,1,15,1012
FM1113,2019-11-09,98,0,20,1490
FM1114,2019-11-12,105,0,19,1432
FM1115,2019-12-05,104,0,22,1659
FM1116,2019-05-25,105,1,13,857


### Rescaling number of transactions and transaction total

In [12]:
x = churned_customers.no_of_tran
churned_customers["scaled_tran"] = (x- x.min())/(x.max() - x.min())
y = churned_customers.tran_total
churned_customers["scaled_amount"] = (y- y.min())/(y.max() - y.min())
churned_customers.head()

,trans_date,tran_amount,churned,no_of_tran,tran_total,scaled_tran,scaled_amount
customer_id,,,,,,,
FM1112,2019-10-14,105,1,15,1012,0.314286,0.309986
FM1113,2019-11-09,98,0,20,1490,0.457143,0.481681
FM1114,2019-11-12,105,0,19,1432,0.428571,0.460848
FM1115,2019-12-05,104,0,22,1659,0.514286,0.542385
FM1116,2019-05-25,105,1,13,857,0.257143,0.254310


### Calculating Churn rate

In [13]:
churned_customers['score'] = round(((0.5*churned_customers['scaled_tran']) + (0.5*churned_customers['scaled_amount'])) * 100, 2)
churned_customers = churned_customers[churned_customers['churned'] == 1].sort_values(by='score', ascending=False)

### Identifying 30 percent of trasaction total for top customers and diving 1000 into them

In [14]:
churned_customers['tran_total_30%'] = round(churned_customers['tran_amount']*.30,0).astype('int')
churned_customers['tran_total_30%'].head(32).sum()

1006

### Best churned customers to retain 

In [15]:
best_churned_customers = churned_customers.head(32)
best_churned_customers

,trans_date,tran_amount,churned,no_of_tran,tran_total,scaled_tran,scaled_amount,score,tran_total_30%
customer_id,,,,,,,,,
FM4320,2019-09-26,105,1,38,2647,0.971429,0.897270,93.43,32
FM3799,2019-07-16,105,1,36,2513,0.914286,0.849138,88.17,32
FM3805,2019-09-11,104,1,35,2453,0.885714,0.827586,85.67,31
FM5752,2019-09-28,102,1,33,2612,0.828571,0.884698,85.66,31
FM4074,2019-09-05,105,1,34,2462,0.857143,0.830819,84.40,32
FM1215,2019-08-03,102,1,35,2362,0.885714,0.794899,84.03,31
FM2620,2019-07-25,104,1,35,2360,0.885714,0.794181,83.99,31
FM1580,2019-06-07,102,1,33,2329,0.828571,0.783046,80.58,31
FM2951,2019-05-28,103,1,32,2382,0.800000,0.802083,80.10,31
